# Practice Create persistent volume and link it to a pod

For this practice, we are going to integrate persisent volumes (data storage) into our pod.
In the lab activity, you simply uploaded and downloaded files to and from the running pod.
This is using ephemeral storage, that is a data storage area that simply disappears when the container / pod shuts down.

##### Make sure you have read the information on persistent volumes linked into this module.

# Part 1: Create persistent volume

## Modify the yml file 

Edit  `resources/persistent_volume.yml`   
Specify the name of your persistent volume in line 4 (replace `your_name`) with your `SSO-pv`

```BASH
kubectl -n gpn-mizzou-tutorial create -f persistent_volume.yml
```

##### Progress Check: Did you see something similar to:
```BASH
jovyan@jupyter-scottgs-40missouri-2eedu:~$ kubectl -n gpn-mizzou-tutorial create -f persistent_volume.yml
persistentvolumeclaim/scottgs-pv created
```


## Check if the persistent volume is created

The command below will list all persitent volumes related to your namespace

```BASH
kubectl -n gpn-mizzou-tutorial get pvc
```

## Attach the persistent volume

  The only way to access persistent volume is by attaching it to a pod. 
  This requires some modification of the yaml file of the pod to be set up

## Modify pod yaml file to mount persistent volume

Open `resources/pod_pvc.yml` in a text editor and perform the following:   
* line  4: Update pod name    
* line 18: This is the folder name that appears when you access your pod   
* line 19: Specify a name of the mounted volume to be recognized by the pod, for simplicity you can use the same name as the persistent volume
* line 21: This needs to be the same as in line 19   
* line 23: Add the persistent volume name as specified in the previous section.      

 


### Run the yml file

##### Take note to update the commands below as appropriate for your names

```BASH
kubectl create -f pod_pvc.yml    
```

#### Confirm the pod is running, then access it.

```
kubectl exec -it pod_name -- /bin/bash
```

#### Once you are in the pod, confirm the file storage is attached

We can use the `df -h` command to confirm that we have a `/data` folder with 50 GB of storage.
Note, that `/data` in this case would be the edit on line 18 of the `pod_pvc.yml`

```BASH
root@pod-scottgs:/# df -h
Filesystem                                                                                                                                                                                   Size  Used Avail Use% Mounted on
overlay                                                                                                                                                                                        300G   38G  263G  13% /
tmpfs                                                                                                                                                                                           64M     0   64M   0% /dev
tmpfs                                                                                                                                                                                          252G     0  252G   0% /sys/fs/cgroup
10.110.132.244:6789,10.107.185.33:6789,10.100.205.16:6789,10.103.88.157:6789,10.98.249.12:6789:/volumes/csi/csi-vol-7e26f559-a07f-11ec-9f90-ae01ed319120/bb0c2797-3dc4-4bc0-abe5-2bd705d8e7d9   50G     0   50G   0% /data
/dev/mapper/k8s-kubelet                                                                                                                                                                         97G  303M   96G   1% /etc/hosts
/dev/mapper/k8s-docker                                                                                                                                                                
```

#### Write some data into the `/data` folder

We will capture the CPU Information for our pod into a text file.
Note, `/data` is what I chose for line 18 of the `pod_pvc.yml`

```BASH
cat /proc/cpuinfo > /data/cpuinfo
```




## Repeat steps from the [Setup and Test kubectl](./activities/SetupAndTest_kubectl.ipynb)  

This time you will find that after:  
* closing the pod  
* deleting the pod  

## Re-spawn the pod, and check that your data is still there

* creating pod again  
* accessing the pod  

#### Test that you data is still there:

Run the command, as before the `/data` should be customized based on your choices.


# Part2: Copy data from one persistent volume to another

Kubernetes allows that to mount more than one persistent volume to one pod. 
This grants us access to both persistent volumea at the same time provided that they are both a part of the same namespace.

We have prepared data for you to use in your exercise but you need to move it to the persistent volume you created.

A pod may not be enough for such task, 
after all pods are meant for simple tasks such as testing and development. 
Instead we will be using a **job**.

## Modify the yaml file for the job

Open `resources/job_copy_data.yml` in a text editor and perform the following:   
* line  4: replace **sso** with your SSO     
* line  9: replace **sso** with your SSO
* line 13: Replace *update* with the name for how your persistent volume will appear when accessing the node
* line 17: Replace *update* with the same name used in line 13
* line 32: Add the name of your persitent volume    
 
  

## Run the job yml file

```BASH
kubectl -n gpn-mizzou-tutorial create -f job_copy_data.yml
```

## Check jobs

```BASH
kubectl -n gpn-mizzou-tutorial get jobs
```


#### Do you see a job with duration?

```BASH
jovyan@jupyter-scottgs-40missouri-2eedu:~$ kubectl -n gpn-mizzou-tutorial get jobs
NAME                        COMPLETIONS   DURATION   AGE
job-data-download-scottgs   0/1           33s        34s
```

## Check pod 

`kubectl -n gpn-mizzou-tutorial get pods`
    
   
#### Progress Check: Do you see a running data-download pod with your SSO?

```BASH
jovyan@jupyter-scottgs-40missouri-2eedu:~$ kubectl -n gpn-mizzou-tutorial get pods
NAME                                 READY   STATUS    RESTARTS   AGE
job-data-download-scottgs--1-626z2   1/1     Running   0          55s
pod-name-sso                         0/1     Error     0          3d17h
```


#### Error Checking:

If you have an error status, you can use the command (replacing pod-name with the NAME from the command above).: 
```
kubectl -n gpn-mizzou-tutorial describe pods pod-name
```

### Check for data copy success

Once you see that the job pob is completed you can proceed.   
Example, note the STATUS is Completed.

```
jovyan@jupyter-scottgs-40missouri-2eedu:~$ kubectl get pods
NAME                                 READY   STATUS      RESTARTS   AGE
job-data-download-scottgs--1-626z2   0/1     Completed   0          7m46s
pod-name-sso                         0/1     Error       0          3d17h
```

* Spin up a pod that has the mount (look up to Part 1).
* Log into the pod and look for data
```BASH
kubectl -n gpn-mizzou-tutorial exec -it pod_name -- /bin/bash    
cd /data    
ls
```




##### Expected Check Output is Similar to this

```BASH
jovyan@jupyter-scottgs-40missouri-2eedu:~$ kubectl -n gpn-mizzou-tutorial exec -it pod-scottgs -- /bin/bash
root@pod-scottgs:/# cd /data/
root@pod-scottgs:/data# ls
U_Net_evaluation.py  U_Net_training.py  create_fold_lists.py  images  lists  models
U_Net_prediction.py  cpuinfo            evaluation            labels  logs   predictions
root@pod-scottgs:/data# 
```



---

# Part 3: Clean Up!

Once you have captured all the necessary screen shots, please clean up the pods and jobs.

**Important** Do not clean up your persistent volume storage, as the data you copied in is **required** for the next part of the workshop!

Here are some hints on clean up, where you will need to update them a little. 

See the Kubernetes documentation.
 * Cleaning up a job or pod: `kubectl -n gpn-mizzou-tutorial delete -f job_copy_data.yml`


---

# Notebook Complete

You have learned how to
 * Create a persistent storage volume
 * Mount persistent storage volumes to pods
 * Configure jobs to execute commands
 